In [4]:
# default_exp eda
%reload_ext autoreload
%autoreload 2

In [5]:
from nbdev.export import create_config

create_config(host='localhost', lib_name='re', user='ian', title='title1', copyright='cp1', description='des1')

# 关系抽取任务简介
https://kexue.fm/archives/6671

关系抽取 (Relation Extraction, RE) 是从自然语言文本中抽取实体及其之间关系的信息技术，是信息检索、智能问答、智能对话等人工智能应用的重要基础，一直受到业界的广泛关注。关系抽取任务涉及命名实体识别、指代消解、关系分类等复杂技术，极具挑战性。

## 样本特点
这是一个“一对多”的抽取+分类任务

通过对人工观察样本情况，发现其特点如下：

    1、s和o未必是分词工具分出来的词，因此要对query做标注才能抽取出正确的s、o，而考虑到分词可能切错边界，因此应该使用基于字的输入来标注；

    2、样本中大多数的抽取结果是“一个s、多个(p, o)”的形式，比如“《战狼》的主演包括吴京和余男”，那么要抽出“(战狼, 主演, 吴京)”、“(战狼, 主演, 余男)”；

    3、抽取结果是“多个s、一个(p, o)”甚至是“多个s、多个(p, o)”的样本也占有一定比例，比如“《战狼》、《战狼2》的主演都是吴京”，那么要抽出“(战狼, 主演, 吴京)”、“(战狼2, 主演, 吴京)”；

    4、同一对(s, o)也可能对应多个p，比如“《战狼》的主演和导演都是吴京”，那么要抽出“(战狼, 主演, 吴京)”、“(战狼, 导演, 吴京)”；

    5、极端情况下，s、o之间是可能重叠的，比如“《鲁迅自传》由江苏文艺出版社出版”，严格上来讲，除了要抽出“(鲁迅自传, 出版社, 江苏文艺出版社)”外，还应该抽取出“(鲁迅自传, 作者, 鲁迅)”。




# 2020语言与智能技术竞赛：关系抽取任务
https://aistudio.baidu.com/aistudio/competition/detail/31

关系抽取 (Relation Extraction, RE) 是从自然语言文本中抽取实体及其之间关系的信息技术，是信息检索、智能问答、智能对话等人工智能应用的重要基础，一直受到业界的广泛关注。关系抽取任务涉及命名实体识别、指代消解、关系分类等复杂技术，极具挑战性。

本次竞赛在去年信息抽取竞赛的基础上进行了升级。整体的任务形式仍然是 schema 约束下的关系抽取，也就是在给定关系集合下，从自然语言文本中抽取出符合关系 schema 约束的 SPO 三元组知识，但是对 O 值形态进行了复杂化的扩展。相信这会给参赛者带来更大的挑战和乐趣。

除此之外，本次竞赛将提供业界规模最大的中文关系抽取数据集 DuIE 2.0，旨在为研究者提供学术交流平台，进一步提升中文关系抽取技术的研究水平，推动相关人工智能应用的发展。
## Agenda

    2020/3/10 	启动竞赛报名，发放样例数据
    2020/3/31 	开放评测入口和排行榜，对报名者发放全部训练数据和第一批测试数据
    2020/5/12 	报名截止
    2020/5/13 	发放最终测试数据
    2020/5/20 	系统结果提交截止
    2020/5/30 	公布竞赛结果，接收系统报告和论文
    2020/6/30 	论文提交截止日期
    2020/7 	在“语言与智能高峰论坛”上交流和颁奖

## 数据介绍 Data

### schema

任务目标是在给定的文本句子中，根据预先定义的schema集合，抽取出所有满足 schema 约束的 SPO 三元组。

schema 定义了
* 关系 P predicate，即两个实体之间的关系
* 以及其对应的主体 S subject，即主实体，为query中的一个片段
* 和客体 O object，即客实体，也是query中的一个片段；根据 O 类型的复杂程度可以划分为以下两种：

1.简单 O 值：也就是说 O 是一个单一的文本。例如，「妻子」关系的 schema 定义为：

    {
        S_TYPE: 人物,
        P: 妻子,
        O_TYPE: {
            @value: 人物
        }
    }

简单 O 值是最常见关系类型，去年竞赛中所发布的所有 schema 都属于这种类型。为了保持格式统一，简单 O 值类型的 schema 定义通过结构体保存，结构体中只有一个 @value 字段存放真正的 O 值类型。

2.复杂 O 值：也就是说 O 是一个结构体，由多个语义明确的文本共同组成，多个文本对应了结构体中的多个槽位（slot）。例如，「饰演」关系中 O 值有两个槽位 @value 和 inWork，分别表示「饰演的角色是什么」以及「在哪部影视作品中发生的饰演关系」，其 schema 定义为：

    {
        S_TYPE: 娱乐人物,
        P: 饰演,
        O_TYPE: {
            @value: 角色
            inWork: 影视作品
        }
    } 

在复杂 O 值类型的定义中，@value 槽位可以认为是该关系的默认 O 值槽位，对于该关系不可或缺，其他槽位均可缺省。

输入/输出：

    输入：schema约束集合及句子sent

    输出： 句子sent中包含的符合给定schema约束的三元组知识Triples

### 数据简介

本次竞赛数据集共包含 
* 48个已定义好的schema

    * Graph1 中展示了DuIE2.0数据集中包含的43个简单知识的schema及对应的例子，
    * Graph2 中展示了DuIE2.0数据集中包含的5个复杂知识的schema及对应的例子。
* 超过21万中文句子，数据集中的句子来自百度百科、百度贴吧和百度信息流文本。其中包括17万训练集，2万验证集和2万测试集，共分为以下5个部分：

1.训练集：共17万个句子，包含句子中对应的SPO，用于竞赛模型训练。

2.验证集：共2万个句子，包含句子中对应的SPO，用于竞赛模型训练和参数调试。

3.schema约束：共48个限定的schema，定义了关系P以及其对应的主体S和客体O的类别。

4.测试集1：约1万个句子，不包含句子中对应的SPO，用于参赛者在平台上自助提交模型预测结果、验证效果。

5.测试集2:本次竞赛最终测试集，约2万个句子，不包含句子对应的SPO，包含测试集1。另外为了防止针对测试集的调试，数据中将会额外加入混淆数据。该部分数据在评测结束前一周发布，结果不能在平台上自助验证，由评测委员会进行离线评测。
### 样例数据 Data Sample

平台提供的数据为JSON文件格式，样例如下:

    {
        "text":"王雪纯是87版《红楼梦》中晴雯的配音者，她是中央台《正大综艺》的主持人",
        "spo_list":[
            {
                "predicate":"配音",
                "subject_type":"娱乐人物",
                "object":{
                    "@value":"晴雯",
                    "inWork":"红楼梦"
                },
                "object_type":{
                    "@value":"人物",
                    "inWork":"影视作品"
                },
                "subject":"王雪纯"
            },
            {
                "predicate":"主持人",
                "subject_type":"电视综艺",
                "object":{
                    "@value":"王雪纯"
                },
                "object_type":{
                    "@value":"人物"
                },
                "subject":"正大综艺"
            }
        ]
    }

更多样例和详细数据格式说明参见数据集中包含的“数据格式说明”文档。
Please refer to the specification in dataset package for details.
### 入门参考

1.基线系统：一个开源的基于schema的信息抽取基线系统，将在3月31日前在比赛网站上发布。
Baseline System: An open sourced baseline for information extraction
https://github.com/PaddlePaddle/Research/tree/master/KG/DuIE_Baseline


## 评测方法 Evaluation

对测试集上参赛者给出的 SPO 结果和人工标注的 SPO 结果进行精准匹配，采用 Precision，Recall 和 F1 值作为评价指标。注意，对于复杂 O 值类型的 SPO，必须所有槽位都精确匹配才认为该 SPO 抽取正确。针对部分文本中存在实体别名的问题，我们使用了百度知识图谱的别名词典来辅助评测。准确率、召回率和F1值的计算公式如下：
![](img/e01.png)

其中：n为测试集中句子个数。最终评测是按照F1值进行排名的。

# baseline by苏剑林
* 百度LIC2020的关系抽取赛道，非官方baseline
* 基于“半指针-半标注”结构
* 文章介绍：https://kexue.fm/archives/7161
* 在第一期测试集上能达到0.68的F1，略低于官方baseline

## 用bert4keras做三元组抽取
https://kexue.fm/archives/7161

### 模型简介
基于Bert的三元组抽取模型结构示意图
![基于Bert的三元组抽取模型结构示意图](img/img01.png)
跟之前的策略一样，模型依然是基于“半指针-半标注”的方式来做抽取，顺序是先抽取s，然后传入s来抽取o、p，不同的只是将模型的整体架构换成了bert：

    1、原始序列转id后，传入bert的编码器，得到编码序列；

    2、编码序列接两个二分类器，预测s；

    3、根据传入的s，从编码序列中抽取出s的首和尾对应的编码向量；

    4、以s的编码向量作为条件，对编码序列做一次条件Layer Norm；

    5、条件Layer Norm后的序列来预测该s对应的o、p。
### 类别失衡

不难想到，用“半指针-半标注”结构做实体抽取时，会面临类别不均衡的问题，因为通常来说目标实体词比非目标词要少得多，所以标签1会比标签0少得多。常规的处理不平衡的方法都可以用，比如focal loss或者人工调节类权重，但这些方法用了之后，阈值就不大好取了。我这里用了一种自以为比较恰当的方法：将概率值做n次方。

In [6]:
# export

import json
import os
import numpy as np
from bert4keras.backend import keras, K, batch_gather
from bert4keras.layers import LayerNormalization
from bert4keras.tokenizers import Tokenizer
from bert4keras.models import build_transformer_model
from bert4keras.optimizers import Adam
from bert4keras.snippets import sequence_padding, DataGenerator
from bert4keras.snippets import open, groupby
# from tensorflow.keras.layers import Input, Dense, Lambda, Reshape
# from tensorflow.keras.models import Model
from keras.layers import Input, Dense, Lambda, Reshape
from keras.models import Model
from tqdm import tqdm

/Users/luoyonggui/anaconda3/envs/tf14/lib/python3.7/site-packages/tensorflow/python/framework/dtypes.py:516: FutureWarning: Passing (type, 1) or '1type' as a synonym of type is deprecated; in a future version of numpy, it will be understood as (type, (1,)) / '(1,)type'.
  _np_qint8 = np.dtype([("qint8", np.int8, 1)])
/Users/luoyonggui/anaconda3/envs/tf14/lib/python3.7/site-packages/tensorflow/python/framework/dtypes.py:517: FutureWarning: Passing (type, 1) or '1type' as a synonym of type is deprecated; in a future version of numpy, it will be understood as (type, (1,)) / '(1,)type'.
  _np_quint8 = np.dtype([("quint8", np.uint8, 1)])
/Users/luoyonggui/anaconda3/envs/tf14/lib/python3.7/site-packages/tensorflow/python/framework/dtypes.py:518: FutureWarning: Passing (type, 1) or '1type' as a synonym of type is deprecated; in a future version of numpy, it will be understood as (type, (1,)) / '(1,)type'.
  _np_qint16 = np.dtype([("qint16", np.int16, 1)])
/Users/luoyonggui/anaconda3/envs/tf14

In [23]:
# !pip install tensorflow -U

     |████████████████████████████████| 175.3MB 3.2MB/s eta 0:00:01    |█████████████████▌              | 95.9MB 1.3MB/s eta 0:01:02     |██████████████████▏             | 99.6MB 1.3MB/s eta 0:01:00
  Using cached https://files.pythonhosted.org/packages/d6/84/759f5dd23fec8ba71952d97bcc7e2c9d7d63bdc582421f3cd4be845f0c98/gast-0.3.3-py2.py3-none-any.whl
  Using cached https://files.pythonhosted.org/packages/1a/8b/4d01ae9a9d50a0bcc7b0b9aae41785d8d9de6fa9bba04dc20b1582181d2d/h5py-2.10.0-cp37-cp37m-macosx_10_6_intel.whl
     |████████████████████████████████| 3.0MB 4.9MB/s eta 0:00:01     |██████████████▋                 | 1.4MB 4.9MB/s eta 0:00:01
     |████████████████████████████████| 460kB 5.9MB/s eta 0:00:01
     |████████████████████████████████| 778kB 5.5MB/s eta 0:00:01
ERROR: tf-nightly 2.1.0.dev20191203 has requirement gast==0.2.2, but you'll have gast 0.3.3 which is incompatible.
ERROR: tensorboard 2.2.1 has requirement grpcio>=1.24.3, but you'll have grpcio 1.20.0 which is incomp

In [24]:
!pip freeze | grep tensorflow

tensorflow==2.2.0
tensorflow-datasets==1.3.0
tensorflow-estimator==2.2.0
tensorflow-metadata==0.15.0


In [3]:
!pip freeze | grep Keras

Keras==2.3.1
Keras-Applications==1.0.8
Keras-Preprocessing==1.1.0


In [4]:
!pip freeze | grep keras

bert4keras==0.7.2


In [8]:
os.path.abspath('../../data/baidu_re')

'/Users/luoyonggui/PycharmProjects/data/baidu_re'

In [9]:
# export

bert_dir = '/Users/luoyonggui/Documents/nlpdata/chinese_roberta_wwm_ext_L-12_H-768_A-12'
config_path = f'{bert_dir}/bert_config.json'
checkpoint_path = f'{bert_dir}/bert_model.ckpt'
dict_path = f'{bert_dir}/vocab.txt'

BASE_DIR = os.path.abspath('../../data/baidu_re')
# 基本信息
maxlen = 256
epochs = 20
batch_size = 16
learning_rate = 2e-5

## load_data

In [10]:
# export

def load_data(filename):
    D = []
    with open(filename) as f:
        for l in f:
            l = json.loads(l)
            d = {'text': l['text'], 'spo_list': []}
            for spo in l['spo_list']:
                for k, v in spo['object'].items():
                    d['spo_list'].append(
                        (spo['subject'], spo['predicate'] + '_' + k, v)
                    )
            D.append(d)
    return D


# 加载数据集
train_data = load_data(os.path.join(BASE_DIR, 'data_origin/train_data/train_data.json'))
valid_data = load_data(os.path.join(BASE_DIR, 'data_origin/dev_data/dev_data.json'))

In [3]:
len(train_data), len(valid_data)

(171293, 20674)

In [4]:
train_data[:1], valid_data[:1]

([{'text': '《邪少兵王》是冰火未央写的网络小说连载于旗峰天下',
   'spo_list': [('邪少兵王', '作者_@value', '冰火未央')]}],
 [{'text': '《步步惊心》改编自著名作家桐华的同名清穿小说《甄嬛传》改编自流潋紫所著的同名小说电视剧《何以笙箫默》改编自顾漫同名小说《花千骨》改编自fresh果果同名小说《裸婚时代》是月影兰析创作的一部情感小说《琅琊榜》是根据海宴同名网络小说改编电视剧《宫锁心玉》，又名《宫》《雪豹》，该剧改编自网络小说《特战先驱》《我是特种兵》由红遍网络的小说《最后一颗子弹留给我》改编电视剧《来不及说我爱你》改编自匪我思存同名小说《来不及说我爱你》',
   'spo_list': [('何以笙箫默', '作者_@value', '顾漫'),
    ('我是特种兵', '改编自_@value', '最后一颗子弹留给我'),
    ('步步惊心', '作者_@value', '桐华'),
    ('甄嬛传', '作者_@value', '流潋紫'),
    ('花千骨', '作者_@value', 'fresh果果'),
    ('裸婚时代', '作者_@value', '月影兰析'),
    ('琅琊榜', '作者_@value', '海宴'),
    ('雪豹', '改编自_@value', '特战先驱'),
    ('来不及说我爱你', '改编自_@value', '来不及说我爱你'),
    ('来不及说我爱你', '作者_@value', '匪我思存')]}])

## 读取schema

In [12]:
# export

# 读取schema
with open(os.path.join(BASE_DIR, 'data_origin/schema.json')) as f:
    id2predicate, predicate2id, n = {}, {}, 0
    predicate2type = {}
    for l in f:
        l = json.loads(l)
        predicate2type[l['predicate']] = (l['subject_type'], l['object_type'])
        for k, _ in sorted(l['object_type'].items()):
            key = l['predicate'] + '_' + k
            id2predicate[n] = key
            predicate2id[key] = n
            n += 1

In [28]:
predicate2id

{'毕业院校_@value': 0,
 '嘉宾_@value': 1,
 '配音_@value': 2,
 '配音_inWork': 3,
 '主题曲_@value': 4,
 '代言人_@value': 5,
 '所属专辑_@value': 6,
 '父亲_@value': 7,
 '作者_@value': 8,
 '上映时间_@value': 9,
 '上映时间_inArea': 10,
 '母亲_@value': 11,
 '专业代码_@value': 12,
 '占地面积_@value': 13,
 '邮政编码_@value': 14,
 '票房_@value': 15,
 '票房_inArea': 16,
 '注册资本_@value': 17,
 '主角_@value': 18,
 '妻子_@value': 19,
 '编剧_@value': 20,
 '气候_@value': 21,
 '歌手_@value': 22,
 '获奖_@value': 23,
 '获奖_inWork': 24,
 '获奖_onDate': 25,
 '获奖_period': 26,
 '校长_@value': 27,
 '创始人_@value': 28,
 '首都_@value': 29,
 '丈夫_@value': 30,
 '朝代_@value': 31,
 '饰演_@value': 32,
 '饰演_inWork': 33,
 '面积_@value': 34,
 '总部地点_@value': 35,
 '祖籍_@value': 36,
 '人口数量_@value': 37,
 '制片人_@value': 38,
 '修业年限_@value': 39,
 '所在城市_@value': 40,
 '董事长_@value': 41,
 '作词_@value': 42,
 '改编自_@value': 43,
 '出品公司_@value': 44,
 '导演_@value': 45,
 '作曲_@value': 46,
 '主演_@value': 47,
 '主持人_@value': 48,
 '成立日期_@value': 49,
 '简称_@value': 50,
 '海拔_@value': 51,
 '号_@value': 52,
 '国籍_@value': 53,
 '官方

In [10]:
predicate2type

{'毕业院校': ('人物', {'@value': '学校'}),
 '嘉宾': ('电视综艺', {'@value': '人物'}),
 '配音': ('娱乐人物', {'inWork': '影视作品', '@value': '人物'}),
 '主题曲': ('影视作品', {'@value': '歌曲'}),
 '代言人': ('企业/品牌', {'@value': '人物'}),
 '所属专辑': ('歌曲', {'@value': '音乐专辑'}),
 '父亲': ('人物', {'@value': '人物'}),
 '作者': ('图书作品', {'@value': '人物'}),
 '上映时间': ('影视作品', {'inArea': '地点', '@value': 'Date'}),
 '母亲': ('人物', {'@value': '人物'}),
 '专业代码': ('学科专业', {'@value': 'Text'}),
 '占地面积': ('机构', {'@value': 'Number'}),
 '邮政编码': ('行政区', {'@value': 'Text'}),
 '票房': ('影视作品', {'inArea': '地点', '@value': 'Number'}),
 '注册资本': ('企业', {'@value': 'Number'}),
 '主角': ('文学作品', {'@value': '人物'}),
 '妻子': ('人物', {'@value': '人物'}),
 '编剧': ('影视作品', {'@value': '人物'}),
 '气候': ('行政区', {'@value': '气候'}),
 '歌手': ('歌曲', {'@value': '人物'}),
 '获奖': ('娱乐人物',
  {'inWork': '作品', 'onDate': 'Date', '@value': '奖项', 'period': 'Number'}),
 '校长': ('学校', {'@value': '人物'}),
 '创始人': ('企业', {'@value': '人物'}),
 '首都': ('国家', {'@value': '城市'}),
 '丈夫': ('人物', {'@value': '人物'}),
 '朝代': 

In [13]:
# export

def search(pattern, sequence):
    """从sequence中寻找子串pattern
    如果找到，返回第一个下标；否则返回-1。
    """
    n = len(pattern)
    for i in range(len(sequence)):
        if sequence[i:i + n] == pattern:
            return i
    return -1

In [14]:
# export

# 建立分词器
tokenizer = Tokenizer(dict_path, do_lower_case=True)

In [13]:
token_ids, segment_ids = tokenizer.encode(
                train_data[1]['text']
            )

In [14]:
s = train_data[1]['spo_list'][0][0]
s

'中国科学院上海药物研究所'

In [15]:
s = tokenizer.encode(s)[0][1:-1]
s

[704, 1744, 4906, 2110, 7368, 677, 3862, 5790, 4289, 4777, 4955, 2792]

In [16]:
s_idx = search(s, token_ids)
s_idx

14

In [17]:
s = (s_idx, s_idx + len(s) - 1)
s

(14, 25)

In [18]:
p = predicate2id[train_data[1]['spo_list'][0][1]]
p

50

In [19]:
start, end = np.array([(1, 14), (2, 24), (3, 34)]).T
start, end

(array([1, 2, 3]), array([14, 24, 34]))

In [20]:
end[end>14]

array([24, 34])

In [15]:
# export

class data_generator(DataGenerator):
    """数据生成器
    
    return:
    [
        batch_token_ids, 
        batch_segment_ids,
        batch_subject_labels, 
        batch_subject_ids,
        batch_object_labels
    ], None
    """
    def __iter__(self, random=False):
        batch_token_ids, batch_segment_ids = [], []
        batch_subject_labels, batch_subject_ids, batch_object_labels = [], [], []
        for is_end, d in self.sample(random):
            token_ids, segment_ids = tokenizer.encode(
                d['text'], max_length=maxlen
            )
            # 整理三元组 {s: [(o, p)]}
            spoes = {}
            for s, p, o in d['spo_list']:
                s = tokenizer.encode(s)[0][1:-1]
                p = predicate2id[p]
                o = tokenizer.encode(o)[0][1:-1]
                s_idx = search(s, token_ids)
                o_idx = search(o, token_ids)
                if s_idx != -1 and o_idx != -1:
                    s = (s_idx, s_idx + len(s) - 1)
                    o = (o_idx, o_idx + len(o) - 1, p)
                    if s not in spoes:
                        spoes[s] = []
                    spoes[s].append(o)
            if spoes:
                # subject标签
                subject_labels = np.zeros((len(token_ids), 2))
                for s in spoes:
                    subject_labels[s[0], 0] = 1
                    subject_labels[s[1], 1] = 1
                # 随机选一个subject 
                # mayi: 这里随机选取subject的方法很奇特，同时生成了负样本，如选到的subject_ids不存在时，object_labels都为0
                start, end = np.array(list(spoes.keys())).T
                start = np.random.choice(start)
                end = np.random.choice(end[end >= start])
                # subject对应的索引位置
                subject_ids = (start, end)
                # 对应的object标签
                object_labels = np.zeros((len(token_ids), len(predicate2id), 2))
                for o in spoes.get(subject_ids, []):
                    object_labels[o[0], o[2], 0] = 1
                    object_labels[o[1], o[2], 1] = 1
                # 构建batch
                batch_token_ids.append(token_ids)
                batch_segment_ids.append(segment_ids)
                batch_subject_labels.append(subject_labels)
                batch_subject_ids.append(subject_ids)
                batch_object_labels.append(object_labels)
                if len(batch_token_ids) == self.batch_size or is_end:
                    batch_token_ids = sequence_padding(batch_token_ids)
                    batch_segment_ids = sequence_padding(batch_segment_ids)
                    batch_subject_labels = sequence_padding(
                        batch_subject_labels, padding=np.zeros(2)
                    )
                    batch_subject_ids = np.array(batch_subject_ids)
                    batch_object_labels = sequence_padding(
                        batch_object_labels,
                        padding=np.zeros((len(predicate2id), 2))
                    )
                    yield [
                        batch_token_ids, batch_segment_ids,
                        batch_subject_labels, batch_subject_ids,
                        batch_object_labels
                    ], None
                    batch_token_ids, batch_segment_ids = [], []
                    batch_subject_labels, batch_subject_ids, batch_object_labels = [], [], []



In [11]:
for (batch_token_ids, batch_segment_ids,batch_subject_labels, batch_subject_ids,batch_object_labels), _ in train_generator.forfit():
    break

NameError: name 'train_generator' is not defined

In [47]:
batch_token_ids.shape

(16, 170)

## build nn

In [48]:
bert.model.inputs

[<tf.Tensor 'Input-Token:0' shape=(?, ?) dtype=float32>,
 <tf.Tensor 'Input-Segment:0' shape=(?, ?) dtype=float32>]

In [51]:
a = bert.model.layers[-2]
a

In [ ]:
a.get_output_at()

In [50]:
type(bert.model.layers[-2])

keras.layers.merge.Add

In [16]:
# export

# 补充输入
subject_labels = Input(shape=(None, 2), name='Subject-Labels') # 其中None为token_ids的长度，由于每批tokenid的长度不一样，所以是变化的
subject_ids = Input(shape=(2,), name='Subject-Ids')  # 标出了Subject在tokenid索引的start和end
object_labels = Input(shape=(None, len(predicate2id), 2), name='Object-Labels')  # 同样None为tokenid的长度

# 加载预训练模型
bert = build_transformer_model(
    config_path=config_path,
    checkpoint_path=checkpoint_path,
    return_keras_model=False,
)

# bert.model.inputs
"""
[<tf.Tensor 'Input-Token:0' shape=(?, ?) dtype=float32>,
 <tf.Tensor 'Input-Segment:0' shape=(?, ?) dtype=float32>]
"""
# 预测subject
output = Dense(
    units=2, activation='sigmoid', kernel_initializer=bert.initializer
)(bert.model.output)
subject_preds = Lambda(lambda x: x**2)(output)

subject_model = Model(bert.model.inputs, subject_preds)

In [17]:
# export

# 传入subject，预测object
# 通过Conditional Layer Normalization将subject融入到object的预测中
output = bert.model.layers[-2].get_output_at(-1)


def extrac_subject(inputs):
    """根据subject_ids从output中取出subject的向量表征
    """
    output, subject_ids = inputs
    subject_ids = K.cast(subject_ids, 'int32')
    start = batch_gather(output, subject_ids[:, :1])
    end = batch_gather(output, subject_ids[:, 1:])
    subject = K.concatenate([start, end], 2)
    return subject[:, 0]

subject = Lambda(extrac_subject)([output, subject_ids])
output = LayerNormalization(conditional=True)([output, subject])
output = Dense(
    units=len(predicate2id) * 2,
    activation='sigmoid',
    kernel_initializer=bert.initializer
)(output)
output = Lambda(lambda x: x**4)(output)
object_preds = Reshape((-1, len(predicate2id), 2))(output)

object_model = Model(bert.model.inputs + [subject_ids], object_preds)

# 训练模型
train_model = Model(
    bert.model.inputs + [subject_labels, subject_ids, object_labels],
    [subject_preds, object_preds]
)
train_model.summary()

mask = bert.model.get_layer('Embedding-Token').output_mask
mask = K.cast(mask, K.floatx())

subject_loss = K.binary_crossentropy(subject_labels, subject_preds)
subject_loss = K.mean(subject_loss, 2)
subject_loss = K.sum(subject_loss * mask) / K.sum(mask)

object_loss = K.binary_crossentropy(object_labels, object_preds)
object_loss = K.sum(K.mean(object_loss, 3), 2)
object_loss = K.sum(object_loss * mask) / K.sum(mask)

train_model.add_loss(subject_loss + object_loss)

optimizer = Adam(learning_rate)
train_model.compile(optimizer=optimizer)

Model: "model_4"
__________________________________________________________________________________________________
Layer (type)                    Output Shape         Param #     Connected to                     
Input-Token (InputLayer)        (None, None)         0                                            
__________________________________________________________________________________________________
Input-Segment (InputLayer)      (None, None)         0                                            
__________________________________________________________________________________________________
Embedding-Token (Embedding)     (None, None, 768)    16226304    Input-Token[0][0]                
__________________________________________________________________________________________________
Embedding-Segment (Embedding)   (None, None, 768)    1536        Input-Segment[0][0]              
____________________________________________________________________________________________

/Users/luoyonggui/anaconda3/envs/tf14/lib/python3.7/site-packages/keras/engine/training_utils.py:819: UserWarning: Output lambda_1 missing from loss dictionary. We assume this was done on purpose. The fit and evaluate APIs will not be expecting any data to be passed to lambda_1.
  'be expecting any data to be passed to {0}.'.format(name))
/Users/luoyonggui/anaconda3/envs/tf14/lib/python3.7/site-packages/keras/engine/training_utils.py:819: UserWarning: Output reshape_1 missing from loss dictionary. We assume this was done on purpose. The fit and evaluate APIs will not be expecting any data to be passed to reshape_1.
  'be expecting any data to be passed to {0}.'.format(name))


## 评估和结果输出

In [18]:
# export

def extract_spoes(text):
    """抽取输入text所包含的三元组
    """
    tokens = tokenizer.tokenize(text, max_length=maxlen)
    mapping = tokenizer.rematch(text, tokens)
    token_ids, segment_ids = tokenizer.encode(text, max_length=maxlen)
    # 抽取subject
    subject_preds = subject_model.predict([[token_ids], [segment_ids]])
    start = np.where(subject_preds[0, :, 0] > 0.4)[0]
    end = np.where(subject_preds[0, :, 1] > 0.4)[0]
    subjects = []
    for i in start:
        j = end[end >= i]
        if len(j) > 0:
            j = j[0]
            subjects.append((i, j))
    if subjects:
        spoes = []
        token_ids = np.repeat([token_ids], len(subjects), 0)
        segment_ids = np.repeat([segment_ids], len(subjects), 0)
        subjects = np.array(subjects)
        # 传入subject，抽取object和predicate
        object_preds = object_model.predict([token_ids, segment_ids, subjects])
        for subject, object_pred in zip(subjects, object_preds):
            start = np.where(object_pred[:, :, 0] > 0.4)
            end = np.where(object_pred[:, :, 1] > 0.4)
            for _start, predicate1 in zip(*start):
                for _end, predicate2 in zip(*end):
                    if _start <= _end and predicate1 == predicate2:
                        spoes.append(
                            ((mapping[subject[0]][0],
                              mapping[subject[1]][-1]), predicate1,
                             (mapping[_start][0], mapping[_end][-1]))
                        )
                        break
        return [(text[s[0]:s[1] + 1], id2predicate[p], text[o[0]:o[1] + 1])
                for s, p, o, in spoes]
    else:
        return []


def combine_spoes(spoes):
    """合并SPO成官方格式
    """
    new_spoes = {}
    for s, p, o in spoes:
        p1, p2 = p.split('_')
        if (s, p1) in new_spoes:
            new_spoes[(s, p1)][p2] = o
        else:
            new_spoes[(s, p1)] = {p2: o}

    return [(k[0], k[1], v) for k, v in new_spoes.items()]


class SPO(tuple):
    """用来存三元组的类
    表现跟tuple基本一致，只是重写了 __hash__ 和 __eq__ 方法，
    使得在判断两个三元组是否等价时容错性更好。
    """
    def __init__(self, spo):
        self.spox = (
            tuple(tokenizer.tokenize(spo[0])),
            spo[1],
            tuple(
                sorted([
                    (k, tuple(tokenizer.tokenize(v))) for k, v in spo[2].items()
                ])
            ),
        )

    def __hash__(self):
        return self.spox.__hash__()

    def __eq__(self, spo):
        return self.spox == spo.spox


def evaluate(data):
    """评估函数，计算f1、precision、recall
    """
    X, Y, Z = 1e-10, 1e-10, 1e-10
    f = open('dev_pred.json', 'w', encoding='utf-8')
    pbar = tqdm()
    for d in data:
        R = combine_spoes(extract_spoes(d['text']))
        T = combine_spoes(d['spo_list'])
        R = set([SPO(spo) for spo in R])
        T = set([SPO(spo) for spo in T])
        X += len(R & T)
        Y += len(R)
        Z += len(T)
        f1, precision, recall = 2 * X / (Y + Z), X / Y, X / Z
        pbar.update()
        pbar.set_description(
            'f1: %.5f, precision: %.5f, recall: %.5f' % (f1, precision, recall)
        )
        s = json.dumps({
            'text': d['text'],
            'spo_list': list(T),
            'spo_list_pred': list(R),
            'new': list(R - T),
            'lack': list(T - R),
        },
                       ensure_ascii=False,
                       indent=4)
        f.write(s + '\n')
    pbar.close()
    f.close()
    return f1, precision, recall


def predict_to_file(in_file, out_file):
    """预测结果到文件，方便提交
    """
    fw = open(out_file, 'w', encoding='utf-8')
    with open(in_file) as fr:
        for l in tqdm(fr):
            l = json.loads(l)
            spoes = combine_spoes(extract_spoes(l['text']))
            spoes = [{
                'subject': spo[0],
                'subject_type': predicate2type[spo[1]][0],
                'predicate': spo[1],
                'object': spo[2],
                'object_type': {
                    k: predicate2type[spo[1]][1][k]
                    for k in spo[2]
                }
            }
                     for spo in spoes]
            l['spo_list'] = spoes
            s = json.dumps(l, ensure_ascii=False)
            fw.write(s + '\n')
    fw.close()


class Evaluator(keras.callbacks.Callback):
    """评估和保存模型
    """
    def __init__(self):
        self.best_val_f1 = 0.

    def on_epoch_end(self, epoch, logs=None):
        f1, precision, recall = evaluate(valid_data)
        if f1 >= self.best_val_f1:
            self.best_val_f1 = f1
            train_model.save_weights('best_model.weights')
        print(
            'f1: %.5f, precision: %.5f, recall: %.5f, best f1: %.5f\n' %
            (f1, precision, recall, self.best_val_f1)
        )

In [19]:
# export

train_generator = data_generator(train_data, batch_size)
evaluator = Evaluator()

In [16]:
train_generator

In [17]:
len(train_generator)

10706

In [18]:
len(train_data)//batch_size + 1

10706

## fit

In [20]:
# export

train_model.fit_generator(
    train_generator.forfit(),
    steps_per_epoch=len(train_generator),
    epochs=epochs,
    callbacks=[evaluator]
)

Instructions for updating:
Use tf.where in 2.0, which has the same broadcast rule as np.where

Epoch 1/20
    3/10706 [..............................] - ETA: 53:42:04 - loss: 4.9064

KeyboardInterrupt: 

In [28]:
# export

train_model.save_weights(os.path.join(BASE_DIR, 'weights.h5'))

In [16]:
# export

# train_model.load_weights('best_model.weights')
predict_to_file(os.path.join(BASE_DIR, 'data_origin/test1_data/test1_data.json', 'ie_pred.json'))

10468it [1:22:59,  2.10it/s]


In [26]:
from nbdev.export import *
notebook2script('0EDA.ipynb')

Converted 0EDA.ipynb.


In [27]:
!python re/eda.py

/Users/luoyonggui/anaconda3/envs/tf14/lib/python3.7/site-packages/tensorflow/python/framework/dtypes.py:516: FutureWarning: Passing (type, 1) or '1type' as a synonym of type is deprecated; in a future version of numpy, it will be understood as (type, (1,)) / '(1,)type'.
  _np_qint8 = np.dtype([("qint8", np.int8, 1)])
/Users/luoyonggui/anaconda3/envs/tf14/lib/python3.7/site-packages/tensorflow/python/framework/dtypes.py:517: FutureWarning: Passing (type, 1) or '1type' as a synonym of type is deprecated; in a future version of numpy, it will be understood as (type, (1,)) / '(1,)type'.
  _np_quint8 = np.dtype([("quint8", np.uint8, 1)])
/Users/luoyonggui/anaconda3/envs/tf14/lib/python3.7/site-packages/tensorflow/python/framework/dtypes.py:518: FutureWarning: Passing (type, 1) or '1type' as a synonym of type is deprecated; in a future version of numpy, it will be understood as (type, (1,)) / '(1,)type'.
  _np_qint16 = np.dtype([("qint16", np.int16, 1)])
/Users/luoyonggui/anaconda3/envs/tf14

/Users/luoyonggui/anaconda3/envs/tf14/lib/python3.7/site-packages/keras/engine/training_utils.py:819: UserWarning: Output lambda_1 missing from loss dictionary. We assume this was done on purpose. The fit and evaluate APIs will not be expecting any data to be passed to lambda_1.
  'be expecting any data to be passed to {0}.'.format(name))
/Users/luoyonggui/anaconda3/envs/tf14/lib/python3.7/site-packages/keras/engine/training_utils.py:819: UserWarning: Output reshape_1 missing from loss dictionary. We assume this was done on purpose. The fit and evaluate APIs will not be expecting any data to be passed to reshape_1.
  'be expecting any data to be passed to {0}.'.format(name))
Instructions for updating:
Use tf.where in 2.0, which has the same broadcast rule as np.where

Epoch 1/20
    5/10706 [..............................] - ETA: 45:22:35 - loss: 4.5277^C
